## Putting it all together

In [ ]:
# Import PyTorch and matplotlib
import torch
from torch import nn # nn contain all of PT building blocks
import matplotlib.pyplot as plt

# Py version
torch.__version__

Device-agnostic code. 
Will use GPU if available


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Data



In [ ]:
# Create toy data using linear regression formula y = weight * X + bias
weight = 0.7
bias = 0.3

# set range of values
start = 0
end = 1
step = 0.02

# Create X and y (features and labels)
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

In [ ]:
# split data
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
    """
    Plots training data, test data and compares predictions.
    """
    plt.figure(figsize=(10,7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

    # if predictions
    if predictions is not None:
        # Plot the predictions if they exist
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")
    
    plt.legend(prop={"size": 14});

In [ ]:
plot_predictions(X_train, y_train, X_test, y_test)

# PyTorch linear model

In [ ]:
# subclass linear model
class LinearRegressionModelWLayers(nn.Module):
    def __init__(self):
        super().__init__()
        # using nn.Linear() for model params
        self.linear_layer = nn.Linear(in_features=1,
                                      out_features=1)
        
        
        
    # forward method to define the computation in model
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)
    
torch.manual_seed(42)
model_1 = LinearRegressionModelWLayers()
model_1, model_1.state_dict()

<!--  -->

In [ ]:
# Check model current device
next(model_1.parameters()).device

In [ ]:
model_1.to(device)
next(model_1.parameters()).device

# Training
requires:
* Loss function
* Optimizer
* Training Loop
* Testing Loop
  

In [ ]:
# Setup loss function
loss_fn = nn.L1Loss()

# Setup optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01 # hyperparameter (something developer sets) learning rate
                            )

In [ ]:
# Training Loop
torch.manual_seed(42)

epochs = 200

# Put data on target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

# tracking model metrics
epoch_count = []
loss_values = []
test_loss_values = []

### Training
for epoch in range(epochs):
    # Set model to training mode
    model_1.train() 

    # 1. Forward pass
    y_pred = model_1(X_train)

    # 2. calc loss
    loss = loss_fn(y_pred, y_train)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. perform backprop
    loss.backward()

    # 5. Step the optimizer
    optimizer.step() 

    # Testing
    model_1.eval() 
    with torch.inference_mode(): # turns off gradient tracking
        # 1. forward pass
        test_pred = model_1(X_test)

        # 2. Calc loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 10 == 0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)
        print(f"Epoch: {epoch} | train Loss: {loss} | Test loss: {test_loss}")

        # print model state
        print(model_1.state_dict())


In [ ]:
model_1.state_dict()

In [ ]:
# Make prediction
model_1.eval()
with torch.inference_mode():
    y_preds = model_1(X_test)

y_preds

In [ ]:
plot_predictions(predictions=y_preds.cpu())

In [ ]:
# Saving PyTorch model
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2.Create model save path
MODEL_NAME = "01_pytorch_workflow_model_1.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save model state dictionary
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_1.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
# 1. Instantiate new model class
load_model_1 = LinearRegressionModelWLayers()

# 2. load saves state_dict
load_model_1.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

load_model_1.to(device)

load_model_1.state_dict()

In [ ]:
# Evaluate loading model
load_model_1.eval()
with torch.inference_mode():
    load_model_preds = load_model_1(X_test)

y_preds == load_model_preds